This notebook contains experiments for:

* Loss functions
* Learning rate decay
* Weight initialization
* Optimizers
* Dropout

#### lincoln imports

In [16]:
import numpy as np

In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import lincoln
from lincoln.numpy.layers import Dense
from lincoln.numpy.losses import SoftmaxCrossEntropy, MeanSquaredError, SoftmaxCrossEntropyComplex
from lincoln.numpy.optimizers import Optimizer, SGD, SGDMomentum
from lincoln.numpy.activations import Sigmoid, Tanh, Linear, ReLU
from lincoln.numpy.network import NeuralNetwork
from lincoln.numpy.train import Trainer
from lincoln.data import mnist
from lincoln.np_utils import softmax

In [19]:
X_train, y_train, X_test, y_test = mnist.load()

In [20]:
num_labels = len(y_train)
num_labels

60000

In [21]:
num_labels = len(y_train)
train_labels = np.zeros((num_labels, 10))
for i in range(num_labels):
    train_labels[i][y_train[i]] = 1

num_labels = len(y_test)
test_labels = np.zeros((num_labels, 10))
for i in range(num_labels):
    test_labels[i][y_test[i]] = 1

In [8]:
# %load_ext autoreload
# %autoreload 2

# Beginning of MNIST Demos

## Data scaled to mean 0, variance 1

In [9]:
X_train, X_test = X_train - np.mean(X_train), X_test - np.mean(X_train)

In [10]:
np.min(X_train), np.max(X_train), np.min(X_test), np.max(X_test)

(-33.318421449829934,
 221.68157855017006,
 -33.318421449829934,
 221.68157855017006)

In [11]:
X_train, X_test = X_train / np.std(X_train), X_test / np.std(X_train)

In [12]:
np.min(X_train), np.max(X_train), np.min(X_test), np.max(X_test)

(-0.424073894391566, 2.821543345689335, -0.424073894391566, 2.821543345689335)

In [13]:
def calc_accuracy_model(model, test_set):
    return print(f'''The model validation accuracy is: 
    {np.equal(np.argmax(model.forward(test_set, inference=True), axis=1), y_test).sum() * 100.0 / test_set.shape[0]:.2f}%''')

## Softmax cross entropy

In [132]:
model = NeuralNetwork(
    layers=[Dense(neurons=89, 
                  activation=Tanh()),
            Dense(neurons=10, 
                  activation=Sigmoid())],
            loss = MeanSquaredError(normalize=False), 
seed=20190119)

trainer = Trainer(model, SGD(0.1))
trainer.fit(X_train, train_labels, X_test, test_labels,
            epochs = 50,
            eval_every = 10,
            seed=20190119,
            batch_size=60);
print()
calc_accuracy_model(model, X_test)

Validation loss after 10 epochs is 0.611
Validation loss after 20 epochs is 0.428
Validation loss after 30 epochs is 0.389
Validation loss after 40 epochs is 0.374
Validation loss after 50 epochs is 0.366
The model validation accuracy is: 72.58%


In [133]:
model = NeuralNetwork(
    layers=[Dense(neurons=89, 
                  activation=Tanh()),
            Dense(neurons=10, 
                  activation=Sigmoid())],
            loss = MeanSquaredError(normalize=True), 
seed=20190119)

trainer = Trainer(model, SGD(0.1))
trainer.fit(X_train, train_labels, X_test, test_labels,
            epochs = 50,
            eval_every = 10,
            seed=20190119,
            batch_size=60);

calc_accuracy_model(model, X_test)

Validation loss after 10 epochs is 0.952
Loss increased after epoch 20, final loss was 0.952, using the model from epoch 10
The model validation accuracy is: 41.73%


In [217]:
model = NeuralNetwork(
    layers=[Dense(neurons=89, 
                  activation=Tanh()),
            Dense(neurons=10, 
                  activation=Linear())],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

trainer = Trainer(model, SGD(0.1))
trainer.fit(X_train, train_labels, X_test, test_labels,
            epochs = 50,
            eval_every = 10,
            seed=20190119,
            batch_size=60);
print()
calc_accuracy_model(model, X_test)

Validation loss after 10 epochs is 0.630
Validation loss after 20 epochs is 0.574
Validation loss after 30 epochs is 0.549
Validation loss after 40 epochs is 0.546
Loss increased after epoch 50, final loss was 0.546, using the model from epoch 40

The model validation accuracy is: 
    91.01%


## SGD Momentum

In [137]:
model = NeuralNetwork(
    layers=[Dense(neurons=89, 
                  activation=Tanh()),
            Dense(neurons=10, 
                  activation=Linear())],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

optim = SGD(0.1)

optim = SGDMomentum(0.1, momentum=0.9)

trainer = Trainer(model, SGDMomentum(0.1, momentum=0.9))
trainer.fit(X_train, train_labels, X_test, test_labels,
            epochs = 50,
            eval_every = 10,
            seed=20190119,
            batch_size=60);

calc_accuracy_model(model, X_test)

Validation loss after 10 epochs is 0.441
Validation loss after 20 epochs is 0.351
Validation loss after 30 epochs is 0.345
Validation loss after 40 epochs is 0.338
Loss increased after epoch 50, final loss was 0.338, using the model from epoch 40
The model validation accuracy is: 95.51%


## Different weight decay

In [232]:
model = NeuralNetwork(
    layers=[Dense(neurons=89, 
                  activation=Tanh()),
            Dense(neurons=10, 
                  activation=Linear())],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

trainer = Trainer(model, SGDMomentum(0.15, momentum=0.9, final_lr = 0.05, decay_type='linear'))
trainer.fit(X_train, train_labels, X_test, test_labels,
            epochs = 50,
            eval_every = 10,
            seed=20190119,
            batch_size=60);

calc_accuracy_model(model, X_test)

Validation loss after 10 epochs is 0.372
Validation loss after 20 epochs is 0.305
Validation loss after 30 epochs is 0.290
Loss increased after epoch 40, final loss was 0.290, using the model from epoch 30
The model validation accuracy is: 
    95.69%


In [226]:
model = NeuralNetwork(
    layers=[Dense(neurons=89, 
                  activation=Tanh()),
            Dense(neurons=10, 
                  activation=Linear())],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

trainer = Trainer(model, SGDMomentum(0.2, momentum=0.9, final_lr = 0.05, decay_type='exponential'))
trainer.fit(X_train, train_labels, X_test, test_labels,
            epochs = 50,
            eval_every = 10,
            seed=20190119,
            batch_size=60);

calc_accuracy_model(model, X_test)

Validation loss after 10 epochs is 0.468
Validation loss after 20 epochs is 0.319
Validation loss after 30 epochs is 0.290
Loss increased after epoch 40, final loss was 0.290, using the model from epoch 30
The model validation accuracy is: 
    95.74%


## Neural network models

### Weight init

In [233]:
model = NeuralNetwork(
    layers=[Dense(neurons=89, 
                  activation=Tanh(),
                  weight_init="glorot"),
            Dense(neurons=10, 
                  activation=Linear(),
                  weight_init="glorot")],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

optimizer = SGDMomentum(0.15, momentum=0.9, final_lr = 0.05, decay_type='linear')

trainer = Trainer(mnist_soft, optimizer)
trainer.fit(X_train, train_labels, X_test, test_labels,
       epochs = 50,
       eval_every = 10,
       seed=20190119,
           batch_size=60,
           early_stopping=True);

calc_accuracy_model(mnist_soft, X_test)

Validation loss after 10 epochs is 0.321
Validation loss after 20 epochs is 0.260
Loss increased after epoch 30, final loss was 0.260, using the model from epoch 20
The model validation accuracy is: 
    95.75%


In [228]:
mnist_soft = NeuralNetwork(
    layers=[Dense(neurons=89, 
                  activation=Tanh(),
                  weight_init="glorot"),
            Dense(neurons=10, 
                  activation=Linear(),
                  weight_init="glorot")],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

trainer = Trainer(mnist_soft, SGDMomentum(0.2, momentum=0.9, final_lr = 0.05, decay_type='exponential'))
trainer.fit(X_train, train_labels, X_test, test_labels,
       epochs = 50,
       eval_every = 10,
       seed=20190119,
           batch_size=60,
           early_stopping=True);

calc_accuracy_model(mnist_soft, X_test)

Validation loss after 10 epochs is 0.410
Validation loss after 20 epochs is 0.261
Validation loss after 30 epochs is 0.244
Loss increased after epoch 40, final loss was 0.244, using the model from epoch 30
The model validation accuracy is: 
    96.52%


## Dropout

In [229]:
model = NeuralNetwork(
    layers=[Dense(neurons=89, 
                  activation=Tanh(),
                  weight_init="glorot",
                  dropout=0.8),
            Dense(neurons=10, 
                  activation=Linear(),
                  weight_init="glorot")],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

trainer = Trainer(model, SGDMomentum(0.2, momentum=0.9, final_lr = 0.05, decay_type='exponential'))
trainer.fit(X_train, train_labels, X_test, test_labels,
       epochs = 50,
       eval_every = 10,
       seed=20190119,
           batch_size=60,
           early_stopping=True);

calc_accuracy_model(model, X_test)

Validation loss after 10 epochs is 0.312
Validation loss after 20 epochs is 0.233
Validation loss after 30 epochs is 0.221
Validation loss after 40 epochs is 0.209
Loss increased after epoch 50, final loss was 0.209, using the model from epoch 40
The model validation accuracy is: 
    96.69%


## Deep Learning, with and without Dropout

In [230]:
model = NeuralNetwork(
    layers=[Dense(neurons=178, 
                  activation=Tanh(),
                  weight_init="glorot",
                  dropout=0.8),
            Dense(neurons=46, 
                  activation=Tanh(),
                  weight_init="glorot",
                  dropout=0.8),
            Dense(neurons=10, 
                  activation=Linear(),
                  weight_init="glorot")],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

trainer = Trainer(model, SGDMomentum(0.2, momentum=0.9, final_lr = 0.05, decay_type='exponential'))
trainer.fit(X_train, train_labels, X_test, test_labels,
       epochs = 50,
       eval_every = 10,
       seed=20190119,
           batch_size=60,
           early_stopping=True);

calc_accuracy_model(model, X_test)

Validation loss after 10 epochs is 0.296
Validation loss after 20 epochs is 0.243
Validation loss after 30 epochs is 0.225
Validation loss after 40 epochs is 0.203
Validation loss after 50 epochs is 0.186
The model validation accuracy is: 
    97.15%


In [237]:
mnist_soft = NeuralNetwork(
    layers=[Dense(neurons=178, 
                  activation=Tanh(),
                  weight_init="glorot",
                  dropout=0.8),
            Dense(neurons=46, 
                  activation=Tanh(),
                  weight_init="glorot",
                  dropout=0.8),
            Dense(neurons=10, 
                  activation=Linear(),
                  weight_init="glorot")],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

trainer = Trainer(mnist_soft, SGDMomentum(0.2, momentum=0.9, final_lr = 0.05, decay_type='exponential'))
trainer.fit(X_train, train_labels, X_test, test_labels,
       epochs = 50,
       eval_every = 10,
       seed=20190119,
           batch_size=60,
           early_stopping=True);

calc_accuracy_model(mnist_soft, X_test)

Validation loss after 10 epochs is 0.296
Validation loss after 20 epochs is 0.243
Validation loss after 30 epochs is 0.225
Validation loss after 40 epochs is 0.203
Validation loss after 50 epochs is 0.186
The model validation accuracy is: 
    97.15%


In [235]:
mnist_soft = NeuralNetwork(
    layers=[Dense(neurons=178, 
                  activation=Tanh(),
                  weight_init="glorot",
                  dropout=0.5),
            Dense(neurons=46, 
                  activation=Tanh(),
                  weight_init="glorot",
                  dropout=0.5),
            Dense(neurons=10, 
                  activation=Linear(),
                  weight_init="glorot")],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

trainer = Trainer(mnist_soft, SGDMomentum(0.2, momentum=0.9, final_lr = 0.05, decay_type='exponential'))
trainer.fit(X_train, train_labels, X_test, test_labels,
       epochs = 100,
       eval_every = 10,
       seed=20190119,
           batch_size=60,
           early_stopping=True);

calc_accuracy_model(mnist_soft, X_test)

Validation loss after 10 epochs is 0.373
Validation loss after 20 epochs is 0.344
Validation loss after 30 epochs is 0.295
Validation loss after 40 epochs is 0.287
Validation loss after 50 epochs is 0.274
Validation loss after 60 epochs is 0.268
Validation loss after 70 epochs is 0.250
Validation loss after 80 epochs is 0.247
Validation loss after 90 epochs is 0.247
Validation loss after 100 epochs is 0.241
The model validation accuracy is: 
    96.40%


In [231]:
mnist_soft = NeuralNetwork(
    layers=[Dense(neurons=178, 
                  activation=Tanh(),
                  weight_init="glorot"),
            Dense(neurons=46, 
                  activation=Tanh(),
                  weight_init="glorot"),
            Dense(neurons=10, 
                  activation=Linear(),
                  weight_init="glorot")],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

trainer = Trainer(mnist_soft, SGDMomentum(0.2, momentum=0.9, final_lr = 0.05, decay_type='exponential'))
trainer.fit(X_train, train_labels, X_test, test_labels,
       epochs = 50,
       eval_every = 10,
       seed=20190119,
           batch_size=60,
           early_stopping=True);

calc_accuracy_model(mnist_soft, X_test)

Validation loss after 10 epochs is 0.517
Validation loss after 20 epochs is 0.334
Validation loss after 30 epochs is 0.249
Validation loss after 40 epochs is 0.247
Loss increased after epoch 50, final loss was 0.247, using the model from epoch 40
The model validation accuracy is: 
    95.98%


#### Final model, 1 epoch

In [14]:
mnist_soft = NeuralNetwork(
    layers=[Dense(neurons=178, 
                  activation=Tanh(),
                  weight_init="glorot",
                  dropout=0.8),
            Dense(neurons=46, 
                  activation=Tanh(),
                  weight_init="glorot",
                  dropout=0.8),
            Dense(neurons=10, 
                  activation=Linear(),
                  weight_init="glorot")],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

trainer = Trainer(mnist_soft, SGDMomentum(0.2, momentum=0.9, final_lr = 0.05, decay_type='exponential'))
trainer.fit(X_train, train_labels, X_test, test_labels,
       epochs = 1,
       eval_every = 1,
       seed=20190119,
           batch_size=60,
           early_stopping=True);

calc_accuracy_model(mnist_soft, X_test)

0 3.2459572910091175
10 3.0664165533972625
20 1.7275436661304933
30 1.222254129130674
40 0.750363636816075
50 0.6610351600420998
60 1.162050793195217
70 0.4025496603492532
80 0.5224687796768276
90 0.7927112829708989
100 0.46852054351815864
Evaluating on test set...
The model validation accuracy is:
                    83.23%
110 0.6904234010572262
120 0.5450971863939684
130 0.8392732860525914
140 0.4926345407538619
150 0.4977110330444994
160 0.562031598100105
170 0.23128330349960383
180 0.9229771764001745
190 0.6285661107097584
200 0.4830428161357536
Evaluating on test set...
The model validation accuracy is:
                    89.47%
210 0.1635097917598027
220 0.3101699306819837
230 0.5915025578652312
240 0.44144427641633366
250 0.7103041237379478
260 0.6281740777786917
270 0.5054715704703846
280 0.5827667541204994
290 0.22115550543592144
300 0.3860174726326829
Evaluating on test set...
The model validation accuracy is:
                    89.77%
310 0.3239039459693302
320 0.59826873

In [15]:
mnist_soft = NeuralNetwork(
    layers=[Dense(neurons=178, 
                  activation=Tanh(),
                  weight_init="glorot",
                  dropout=0.8),
            Dense(neurons=46, 
                  activation=Tanh(),
                  weight_init="glorot",
                  dropout=0.8),
            Dense(neurons=10, 
                  activation=Linear(),
                  weight_init="glorot")],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

trainer = Trainer(mnist_soft, SGDMomentum(0.1, momentum=0.9, final_lr = 0.05, decay_type='exponential'))
trainer.fit(X_train, train_labels, X_test, test_labels,
       epochs = 1,
       eval_every = 1,
       seed=20190119,
           batch_size=60,
           early_stopping=True);

calc_accuracy_model(mnist_soft, X_test)

Evaluating on test set...
100 The model validation accuracy is:
                    87.86%
Evaluating on test set...
200 The model validation accuracy is:
                    91.03%
Evaluating on test set...
300 The model validation accuracy is:
                    92.43%
Evaluating on test set...
400 The model validation accuracy is:
                    93.29%
Evaluating on test set...
500 The model validation accuracy is:
                    93.60%
Evaluating on test set...
600 The model validation accuracy is:
                    93.82%
Evaluating on test set...
700 The model validation accuracy is:
                    94.30%
Evaluating on test set...
800 The model validation accuracy is:
                    94.61%
Evaluating on test set...
900 The model validation accuracy is:
                    93.91%
Validation loss after 1 epochs is 0.333
The model validation accuracy is: 
    94.44%
